In [1]:
# Load necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Audio
sound_file = './sound/beep-01a.wav'

In [2]:
df = pd.read_csv('FACTDATA_MAR2017.txt')
# fix datatypes
column_names = df.columns.values
column_datatypes = {}
for i in range(16):
    column_datatypes[column_names[i]] = str
column_datatypes[column_names[16]] = np.int64
for i in range(17, 19):
    column_datatypes[column_names[i]] = np.float64
#print len(column_datatypes)
# Reload dataset
df = pd.read_csv('FACTDATA_MAR2017.txt', dtype=column_datatypes)
# Clean data
df = df[(df.WORKSCH != '*') &
        (df.SALLVL != 'Z') &
        (df.PATCO != '9') &
        (df.AGELVL != 'Z') & 
        (df.LOSLVL != 'Z') & 
        (df.SUPERVIS != '*') &
        (df.GSEGRD != '**') &
        (df.TOA != '**') &
        (df.TOA != '99') &
        (df.EDLVL != '**') & 
        (df.EDLVL.notnull())]
# Prepare data
# Age Level
# Print age level dataframe
age_level = pd.read_csv('DTagelvl.txt')
age_lvls = [chr(num) for num in range(ord('A'), ord('A') + 11)]
age_lvlt = 17
age_dict = {}
for age_lvl in age_lvls:
    age_dict[age_lvl] = age_lvlt
    age_lvlt += 5
df['AGE'] = df['AGELVL'].map(lambda x: age_dict[x])
# Educational Level
# Print educational level dataframe
edu_level = pd.read_csv('DTedlvl.txt')
# Take the EDLVL and map to values in the EDLVLTYP column
# Build dictionary of values
edlvl_dict = {}
edlvls_available = np.unique(df['EDLVL'])
for edlvl in edlvls_available:
    edlvl_dict[edlvl] = int(edu_level[edu_level.EDLVL == edlvl].iloc[0].EDLVLTYP)

get_edlvl_type = lambda x: edlvl_dict[x]
df['EDLVLTYP'] = df['EDLVL'].map(get_edlvl_type)
# Supervisory Status
# Print supervisory status dataframe
supervis_status = pd.read_csv('DTsuper.txt')
# Take the SUPERVIS and map to values in the SUPERTYP column
# Build dictionary of values
supervis_dict = {}
supervis_available = np.unique(df['SUPERVIS'])
for supervis in supervis_available:
    # 3 for supervisor, 2 for leader and 1 for non-supervisor
    supervis_dict[supervis] = 4 - int(supervis_status[supervis_status.SUPERVIS == supervis].iloc[0].SUPERTYP)

get_supervis_type = lambda x: supervis_dict[x]
df['SUPERTYP'] = df['SUPERVIS'].map(get_supervis_type)
# Work schedule
# Print work schedule dataframe
work_schedule = pd.read_csv('DTwrksch.txt')
# Take the WORKSCH and map to values in the WSTYP column
# Build dictionary of values
worksch_dict = {}
worksch_available = np.unique(df['WORKSCH'])
for worksch in worksch_available:
    # 2 for full-time and 1 for part-time
    worksch_dict[worksch] = 3 - int(work_schedule[work_schedule.WORKSCH == worksch].iloc[0].WSTYP)

get_worksch_type = lambda x: worksch_dict[x]
df['WSTYP'] = df['WORKSCH'].map(get_worksch_type)
numeric_features = ['AGE', 'EDLVLTYP', 'SUPERTYP', 'WSTYP', 'SALARY', 'LOS']
numeric_features_df = df[numeric_features]

D:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3,4,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

scaled_data = scale(numeric_features_df)
cluster = KMeans(init='k-means++', n_clusters=scaled_data.shape[1], n_init=10).fit(scaled_data)

In [16]:
# Find out the best number of clusters
#pca = PCA(n_components=scaled_data.shape[1]).fit(scaled_data)
cases = [
    (KMeans, 'k-means++', {}, 'k-means++'),
    (KMeans, 'random', {}, 'random')
    #(KMeans, pca.components_, {}, 'pca')
]

plots = []
legends = []

n_runs = 5
n_clusters_range = range(2, scaled_data.shape[1])

X = scaled_data
for factory, init, params, init_name in cases:
    print ('Evaluation of {} with {} init'.format(factory.__name__, init_name))
    inertia = np.empty((len(n_clusters_range), n_runs))
    
    for run_id in range(n_runs):
        for i, n_clusters in enumerate(n_clusters_range):
            print n_clusters
            km = factory(n_clusters = n_clusters, init=init, random_state = run_id, n_init=10).fit(X)
            inertia[i, run_id] = km.inertia_
    p = plt.errorbar(n_clusters_range, inertia.mean(axis=1), inertia.std(axis=1))
    plots.append(p[0])
    legends.append("{} with {} init".format(factory.__name__, init_name))
    
plt.xlabel('n_clusters')
plt.ylabel('inertia')
plt.legend(plots, legends)
plt.title('Mean inertia for various number of clusters across {} runs'.format(n_runs))

Evaluation of KMeans with k-means++ init
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
Evaluation of KMeans with random init
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
2
3
4
5
Evaluation of KMeans with pca init
2


ValueError: The shape of the initial centers ((6L, 6L)) does not match the number of clusters 2

** On second thoughts, the results aren't that useful because it will certainly be the case that the more clusters one gets, the less the inertia will be. **

In [ ]:
km = 